<a href="https://colab.research.google.com/github/Gab2697/Surface-classification-EMG-IMU/blob/main/CNN/Sensor_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and functions

In [1]:
#import
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy.io import loadmat

from keras import layers 
from keras import models
from keras import regularizers
import glob

from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
#================================mat to tensor================================
def mat_to_tensor(path_mat):
  data=loadmat(path_mat)
  keys=list(data['data'].dtype.fields.keys())
  da=np.array(data['data'][keys[0]])
  for i in range(1,len(keys)-2):
      da=np.append(da,data['data'][keys[i]],axis=1)
  da=np.append(da,data['data'][keys[-2]],axis=1)
  da=np.append(da,data['data'][keys[-1]],axis=1)
  da=pd.DataFrame(da,columns=keys)
  Participant= da['Subject']
  Surface= da['Conditions'] 
  # from 2d to 3d
  Ndata=da
  Parameters=Ndata.keys()
  GC_len=Ndata.iloc[1,1].shape[0]
  data1=np.zeros([Ndata.shape[0],GC_len,len(Parameters)-2])
  for i in range(len(Parameters)-2):
    for j in range(len(Ndata[Parameters[i]])):
      data1[j,:,i]=Ndata[Parameters[i]].to_numpy()[j].reshape(GC_len)
  return data1,Participant,Surface,Parameters
  
#================================Open pickle file================================
def open_pickle(path_pickle):
    f = open(path_pickle, 'rb')
    T = pickle.load(f)
    f.close()

    return T

#================================Seperate signals from IMU data================================
def seperate_signals_IMU(type_signal,ALL_data):
  if type_signal=='gyro':
    ALL_data=ALL_data[:,:,0:36]
  elif type_signal=='acc':
    ALL_data=ALL_data[:,:,75:-2]

  return ALL_data

#================================Seperate signals from merged data================================
def seperate_signals_merged(type_signal,ALL_data):
  if type_signal=='gyro':
    ALL_data=ALL_data[:,:,0:36]
  elif type_signal=='acc':
    ALL_data=ALL_data[:,:,36:83]
  elif type_signal=='emg':
    ALL_data=ALL_data[:,:,83:108]
  return ALL_data

#EMG DATA GC- No sweat

In [51]:
#open pickle
ALL_data_NS=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_data_NS.pkl')
ALL_Surfaces_NS=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_Surfaces_NS.pkl')
ALL_Participants_numb_NS=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_Participants_numb_NS.pkl')

In [54]:
#seperate emg from all merged data
ALL_data_emg=seperate_signals_merged('emg',ALL_data_NS)

In [63]:
#================================Seperate EMG options================================
def seperate_EMG_signals(type_signal,ALL_data):
  if type_signal=='EMG_raw':
    ALL_data=ALL_data_emg[:,:,0:4]
  elif type_signal=='EMG_HF':
    ALL_data=ALL_data_emg[:,:,4:8]
  elif type_signal=='EMG_LF':
    ALL_data=ALL_data_emg[:,:,8:13]
  elif type_signal=='EMG_rectified':
    ALL_data=ALL_data_emg[:,:,13:18]
  elif type_signal=='EMG_rms':
    ALL_data=ALL_data_emg[:,:,18:22]
  elif type_signal=='EMG_Env':
    ALL_data=ALL_data_emg[:,:,22:25]
  return ALL_data

In [65]:
# 4 signals of only the raw data
ALL_data_emg_raw=seperate_EMG_signals('EMG_raw',ALL_data_emg) 
#individual muscles
TA=ALL_data_emg_raw[:,:,0]
Gasrtroc=ALL_data_emg_raw[:,:,1]
RF=ALL_data_emg_raw[:,:,2]
BF=ALL_data_emg_raw[:,:,3]

#IMU DATA GC (NOT from merged data)

## Gyroscope data


In [46]:
ALL_data_gyro=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_data_gyro.pkl')
ALL_Surfaces=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_Surfaces.pkl')
ALL_Participants_numb=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_Participants_numb.pkl')

In [38]:
Parameters[0:36]

Index(['SegmentAngularVelocity_Pelvis_x', 'SegmentAngularVelocity_Pelvis_y',
       'SegmentAngularVelocity_Pelvis_z',
       'SegmentAngularVelocity_Right_Shoulder_x',
       'SegmentAngularVelocity_Right_Shoulder_y',
       'SegmentAngularVelocity_Right_Shoulder_z',
       'SegmentAngularVelocity_Right_Forearm_x',
       'SegmentAngularVelocity_Right_Forearm_y',
       'SegmentAngularVelocity_Right_Forearm_z',
       'SegmentAngularVelocity_Right_Hand_x',
       'SegmentAngularVelocity_Right_Hand_y',
       'SegmentAngularVelocity_Right_Hand_z',
       'SegmentAngularVelocity_Left_Shoulder_x',
       'SegmentAngularVelocity_Left_Shoulder_y',
       'SegmentAngularVelocity_Left_Shoulder_z',
       'SegmentAngularVelocity_Left_Forearm_x',
       'SegmentAngularVelocity_Left_Forearm_y',
       'SegmentAngularVelocity_Left_Forearm_z',
       'SegmentAngularVelocity_Left_Hand_x',
       'SegmentAngularVelocity_Left_Hand_y',
       'SegmentAngularVelocity_Left_Hand_z',
       'SegmentAngul

In [69]:
#================================Seperate gyro locations================================
def seperate_Gyro_signals(type_signal,ALL_data): #seven sensors
  if type_signal=='pelvis':
    ALL_data=ALL_data[:,:,0:3]
  elif type_signal=='right_hand':
    ALL_data=ALL_data[:,:,9:12]
  elif type_signal=='left_hand':
    ALL_data=ALL_data[:,:,18:21]
  elif type_signal=='right_lower_leg':
    ALL_data=ALL_data[:,:,21:24]
  elif type_signal=='right_foot':
    ALL_data=ALL_data[:,:,24:27]
  elif type_signal=='left_lower_leg':
    ALL_data=ALL_data[:,:,30:33]
  elif type_signal=='left_foot':
    ALL_data=ALL_data[:,:,33:36]
  return ALL_data

In [71]:
ALL_data_pelvis=seperate_Gyro_signals('pelvis',ALL_data_gyro)
ALL_data_pelvis

<tf.Tensor: shape=(45024, 101, 3), dtype=float64, numpy=
array([[[ 0.14543989, -0.00869755,  0.11431923],
        [ 0.12224121,  0.10967154, -0.17790773],
        [ 0.08433607,  0.15815113, -0.43876639],
        ...,
        [ 0.28992188,  0.50817191,  1.37752493],
        [ 0.45481752,  0.5504017 ,  1.3813469 ],
        [ 0.44017479,  0.49617832,  1.35676095]],

       [[-0.1334017 , -0.66557628, -1.23538555],
        [ 0.11154364, -0.6241136 , -0.78559142],
        [ 0.24731627, -0.65675074, -0.54759963],
        ...,
        [-0.36732847, -0.63769836, -1.84030346],
        [-0.30065694, -0.64771822, -1.40614572],
        [-0.21205965, -0.51223887, -1.15743909]],

       [[-0.21205965, -0.51223887, -1.15743909],
        [-0.16890124, -0.37719603, -1.21006984],
        [-0.10758459, -0.25471137, -1.25437289],
        ...,
        [-0.19579747,  0.31955202, -1.8209067 ],
        [-0.91121175,  0.24120687, -2.49808864],
        [-1.54106134,  0.22447588, -1.35461542]],

       ...,

   

## Acceleration data

In [47]:
ALL_data_acc=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_data_acc.pkl')
ALL_Surfaces=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_Surfaces.pkl')
ALL_Participants_numb=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/IMU/ALL_Participants_numb.pkl')

In [45]:
Parameters[75:-2]

Index(['SensorFreeAcceleration_Pelvis_x', 'SensorFreeAcceleration_Pelvis_y',
       'SensorFreeAcceleration_Pelvis_z', 'SensorFreeAcceleration_Head_x',
       'SensorFreeAcceleration_Head_y', 'SensorFreeAcceleration_Head_z',
       'SensorFreeAcceleration_Right_Shoulder_x',
       'SensorFreeAcceleration_Right_Shoulder_y',
       'SensorFreeAcceleration_Right_Shoulder_z',
       'SensorFreeAcceleration_Right_Upper_Arm_x',
       'SensorFreeAcceleration_Right_Upper_Arm_y',
       'SensorFreeAcceleration_Right_Upper_Arm_z',
       'SensorFreeAcceleration_Right_Forearm_x',
       'SensorFreeAcceleration_Right_Forearm_y',
       'SensorFreeAcceleration_Right_Forearm_z',
       'SensorFreeAcceleration_Right_Hand_x',
       'SensorFreeAcceleration_Right_Hand_y',
       'SensorFreeAcceleration_Right_Hand_z',
       'SensorFreeAcceleration_Left_Shoulder_x',
       'SensorFreeAcceleration_Left_Shoulder_y',
       'SensorFreeAcceleration_Left_Shoulder_z',
       'SensorFreeAcceleration_Left_Uppe

In [67]:
#================================Seperate acceleration locations================================
def seperate_Acc_signals(type_signal,ALL_data):
  if type_signal=='pelvis':
    ALL_data=ALL_data[:,:,0:3]
  elif type_signal=='head':
    ALL_data=ALL_data[:,:,3:6]
  elif type_signal=='right_hand':
    ALL_data=ALL_data[:,:,15:18]
  elif type_signal=='left_hand':
    ALL_data=ALL_data[:,:,27:30]
  elif type_signal=='right_lower_leg':
    ALL_data=ALL_data[:,:,33:36]
  elif type_signal=='right_foot':
    ALL_data=ALL_data[:,:,36:39]
  elif type_signal=='left_lower_leg':
    ALL_data=ALL_data[:,:,42:45]
  elif type_signal=='left_foot':
    ALL_data=ALL_data[:,:,45:48]
  return ALL_data

In [68]:
ALL_data_left_foot=seperate_Acc_signals('left_foot',ALL_data_acc)
ALL_data_left_foot

<tf.Tensor: shape=(45024, 101, 3), dtype=float64, numpy=
array([[[-1.08536526e+01,  1.45401147e+00, -1.04431220e+01],
        [-1.01404660e+01, -5.58895969e-01, -1.32171151e+01],
        [-7.49725740e+00, -2.45849305e+00, -1.36969807e+01],
        ...,
        [-2.45466125e+00, -5.46830487e-01, -5.11505484e-01],
        [-3.69482065e+00,  5.38870314e-01, -5.03506260e-01],
        [-4.21732720e+00,  1.45617420e+00, -4.72367625e-02]],

       [[ 3.60486822e+01, -1.85678767e+00, -1.05885091e+01],
        [ 3.34380815e+01, -2.88121735e+00, -1.28259187e+01],
        [ 3.03055050e+01, -4.27519862e+00, -1.47599301e+01],
        ...,
        [ 3.69415888e+01, -5.76096569e+00, -7.00468226e+00],
        [ 3.36503717e+01, -5.74432620e+00, -1.09605467e+01],
        [ 3.17347230e+01, -4.42403582e+00, -1.40063456e+01]],

       [[ 3.17347230e+01, -4.42403582e+00, -1.40063456e+01],
        [ 3.19370032e+01, -3.14792601e+00, -1.51500565e+01],
        [ 3.19803430e+01, -2.29810642e+00, -1.60925488e+01]